### Backend for Vision Zero Code ###
by Richard Sowers and Lucas Gong

* <r-sowers@illinois.edu>
* <shuog2@illinois.edu>

<https://publish.illinois.edu/r-sowers/>

Copyright 2018 University of Illinois Board of Trustees. All Rights Reserved. Licensed under the MIT license

### imports ###

In [1]:
import osmnx
import networkx
import pandas
import numpy
import time
import itertools
import pytz

### setup ###

In [2]:
tz="America/New_York"
region=pytz.timezone(tz)

### Set up OSMNX ###

In [3]:
osmnx.config(log_file=True, log_console=True, use_cache=True)
G_raw = osmnx.graph_from_place('Manhattan Island, New York, USA', network_type='drive')

INFO:osmnx:Configured osmnx
INFO:osmnx:Pausing 1.00 seconds before making API GET request
INFO:osmnx:Requesting https://nominatim.openstreetmap.org/search?format=json&limit=1&dedupe=0&polygon_geojson=1&q=Manhattan+Island%2C+New+York%2C+USA with timeout=30
INFO:osmnx:Downloaded 84.0KB from nominatim.openstreetmap.org in 0.71 seconds
INFO:osmnx:Saved response to cache file "cache/d20b522fd82b6a936766e24673de3f23.json"
INFO:osmnx:Created GeoDataFrame with 1 row for query "Manhattan Island, New York, USA"
INFO:osmnx:Constructed place geometry polygon(s) to query API
INFO:osmnx:Projected the GeoDataFrame "geometry to project" to UTM-18 in 0.01 seconds
INFO:osmnx:Projected the GeoDataFrame "geometry to project" to EPSG 4326 in 0.00 seconds
INFO:osmnx:Projected the GeoDataFrame "geometry to project" to UTM-18 in 0.00 seconds
INFO:osmnx:Projected the GeoDataFrame "geometry to project" to EPSG 4326 in 0.00 seconds
INFO:osmnx:Requesting network data within polygon from API in 1 request(s)
INFO:o

### get rid of some double ways ###

In [4]:
accident_file = "collisions"
traveltime_file = "travel_times_2013_250000lines"

accidents_raw=pandas.read_pickle(accident_file+".p")
traveltimes_raw=pandas.read_pickle(traveltime_file+".p")

In [5]:
def simplify(in_Dict):
    return {"length": in_Dict[0]["length"]}

G=networkx.DiGraph()
temp=[(begin,end,simplify(G_raw[begin][end])) for (begin,end) in G_raw.edges(keys=False)]
G.add_edges_from(temp)

### load accident file and include in graph ###

In [6]:
accidents=accidents_raw.copy()
accidents.set_index("node",drop=True,inplace=True)
accidents=accidents.groupby(level="node").size()
accidents.head()

node
42421728    12
42421731    12
42421737    38
42421741    21
42421749     9
dtype: int64

In [7]:
networkx.set_edge_attributes(G,0,"accidents") #set default value of zero accidents
for node,count in accidents.iteritems():
    if node not in G.nodes: # discard if node is not in graph
        continue
    for begin,end in G.in_edges(node):
        G[begin][end]["accidents"]=count

### load traveltimes and include in graph ###

In [8]:
traveltimes=pandas.DataFrame(traveltimes_raw)
traveltimes.set_index(["begin_node","end_node"],drop=True,inplace=True)
traveltimes.head()

datetime  travel_time
begin_node end_node                                 
42421728   42435337 2013-12-31 23:00:00      20.4412
           42432736 2013-12-31 23:00:00      24.6495
           42435337 2013-12-31 22:00:00      10.5158
           42432736 2013-12-31 22:00:00      13.0787
           42435337 2013-12-31 21:00:00      11.4267

In [9]:
traveltimes_means=traveltimes["travel_time"].groupby(
    level=["begin_node","end_node"]).mean()

networkx.set_edge_attributes(G,numpy.inf,"time") #set default value of infinite time. 
for edge,traveltime in traveltimes_means.items():
    if edge not in G.edges: # discard if edge is not in graph
        continue
    begin,end=edge
    G[begin][end]["time"]=traveltime

In [10]:
def makePath(origin_node,destination_node,alpha):
    for begin,end in G.edges:
        G[begin][end]["cost"]=(1-alpha)*G[begin][end]["time"]+alpha*G[begin][end]["accidents"]
        
    return networkx.shortest_path(G,source=origin_node,target=destination_node,weight="cost")

        
(origin,destination)=(42435654,42454325)
print(makePath(origin,destination,0.5))

[42435654, 42435657, 42439968, 42428277, 42428272, 42428268, 42428264, 42428258, 42430311, 42452973, 42434946, 42437644, 42446701, 42443680, 42451674, 42437686, 42427786, 42438544, 42434800, 42429874, 42453624, 449581627, 42428201, 42428198, 42428192, 42428183, 42428179, 42428174, 42428170, 42438785, 42431491, 42431490, 42430255, 42439826, 42435598, 387180916, 4557517550, 4557517554, 42454325]


### test ###

In [11]:
(origin,destination)=(42438971, 42452956)
print(makePath(origin,destination,0.25))
print(" ")
print(makePath(origin,destination,0.75))

[42438971, 42443061, 42443058, 42443056, 42443054, 42434175, 42443052, 42443050, 42443048, 42443046, 42443044, 42436549, 42443042, 42443040, 42429340, 42443037, 42435516, 42443032, 42439406, 42443029, 42438506, 42438509, 42452956]
 
[42438971, 42443061, 42443058, 42443056, 42443054, 42434175, 42443052, 42443050, 42443048, 42443046, 42443044, 42444820, 42428047, 42428045, 42428043, 42445757, 42451773, 42429348, 42451766, 42435522, 42448257, 42439416, 42451754, 42428024, 42452956]


### for some (O,D) pairs, the quickest path is also accident-free ##

In [13]:
(origin,destination)=(42435654,42454325)
print(makePath(origin,destination,0.25))
print(" ")
print(makePath(origin,destination,0.75))

[42435654, 42435657, 42439968, 42439964, 42433611, 42428268, 42428264, 42428258, 42430311, 42452973, 42434946, 42437644, 42446701, 42443680, 42451674, 42437686, 42427786, 42438544, 42434800, 42429874, 42453624, 449581627, 42428201, 42428198, 42428192, 42428183, 42428179, 42428174, 42428170, 42438785, 42431491, 42431490, 42430255, 42439826, 42435598, 387180916, 4557517550, 4557517554, 42454325]
 
[42435654, 42435657, 42439968, 42428277, 42428272, 42428268, 42428264, 42428258, 42430311, 42452973, 42434946, 42437644, 42446701, 42443680, 42451674, 42437686, 42427786, 42438544, 42434800, 42429874, 42453624, 449581627, 42428201, 42428198, 42428192, 42428183, 42428179, 42428174, 42428170, 42438785, 42431491, 42431490, 42430255, 42439826, 42435598, 387180916, 4557517550, 4557517554, 42454325]
